# Optimize Initial Conditions
## Brute Force

In [1]:
from scipy.optimize import brute
import numpy as np
from datetime import datetime,timedelta
import pandas as pd

In [2]:
# Load solver
import ray
import LearnerICNoRay as L 

In [3]:
# Initial parameters
dfparam = pd.read_csv("data/param.csv")
dfparam

,state,start-date,prediction-range,s0,e0,a0,i0,r0,d0,START,RATIO,WCASES,WREC
0,SP,2020-03-15,200,8000000.0,0,0,800,300,250,100,0.15,0.55,0.01
1,ES,2020-04-01,200,475000.0,0,0,50,250,50,100,0.15,0.65,0.01
2,MG,2020-04-01,200,1100000.0,0,0,200,250,40,100,0.15,0.55,0.01
3,RJ,2020-03-20,200,700000.0,0,0,800,250,50,100,0.15,0.50,0.01
4,CE,2020-03-20,200,800000.0,0,0,800,250,50,100,0.15,0.50,0.01
5,PE,2020-03-20,200,700000.0,0,0,800,250,100,100,0.15,0.50,0.01
6,AM,2020-03-20,200,700000.0,0,0,800,250,100,100,0.15,0.50,0.01


In [4]:
df = pd.read_csv("data/dados_total_estados.csv")
df

,index,date,state,city,place_type,confirmed,deaths,order_for_place,is_last,popEst,city_ibge_code,confirmed_per_100k_inhabitants,death_rate
0,0,2020-06-06,SE,TOTAL,state,8972,208,74,True,2298696.0,28.0,390.30824,0.0232
1,1,2020-06-05,SE,TOTAL,state,8577,198,73,False,2298696.0,28.0,373.12459,0.0231
2,2,2020-06-04,SE,TOTAL,state,8200,186,72,False,2298696.0,28.0,356.72399,0.0227
3,3,2020-06-03,SE,TOTAL,state,7989,180,71,False,2298696.0,28.0,347.54487,0.0225
4,4,2020-06-02,SE,TOTAL,state,7555,172,70,False,2298696.0,28.0,328.66460,0.0228
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2244,158210,2020-03-11,BA,TOTAL,state,3,0,5,False,14873064.0,29.0,0.02017,0.0000
2245,158211,2020-03-10,BA,TOTAL,state,2,0,4,False,14873064.0,29.0,0.01345,0.0000
2246,158212,2020-03-09,BA,TOTAL,state,2,0,3,False,14873064.0,29.0,0.01345,0.0000
2247,158213,2020-03-07,BA,TOTAL,state,2,0,2,False,14873064.0,29.0,0.01345,0.0000


In [5]:
@ray.remote
def opt(state,e0,a0,r0,d0,date,startNCases, ratio, predict_range, version="1"):
    rranges = [slice(1e5,15e6,1e5),slice(-2,2,4),slice(0,1000,1000),slice(0.1,0.6,0.1),slice(0.1,0.2,0.05)]
    optimal = brute(fobjective,        
        ranges=rranges,
        args=(state,e0,a0,r0,d0,date,startNCases, ratio, predict_range,version), \
        full_output=True, disp=True, finish=None)
    
    return optimal.x

In [6]:
from scipy.integrate import odeint
from itertools import (takewhile,repeat)
import os
import sys
import io

def rawincount(filename):
    f = open(filename, 'rb')
    bufgen = takewhile(lambda x: x, (f.raw.read(1024*1024) for _ in repeat(None)))
    return sum( buf.count(b'\n') for buf in bufgen )

def append_new_line(file_name, text_to_append):
    #Append given text as a new line at the end of file
    # Open the file in append & read mode ('a+')
    with open(file_name, "a+") as file_object:
        # Move read cursor to the start of file.
        file_object.seek(0)
        # If file is not empty then append '\n'
        data = file_object.read(9999999)
        if len(data) > 0:
            file_object.write("\n")
        # Append text at the end of file
        file_object.write(text_to_append)

def fobjective(point,state,e0,a0,r0,d0,date,startNCases, ratio, predict_range, version="1"):
    results=[]
    s0, deltaDate, i0, weigthCases, weigthRecov = point

    s0=int(round(s0))
    i0=int(round(i0))
    deltaDate=int(deltaDate)
    
    cleanRecovered = False
    
    Date = datetime.strptime(date, "%Y-%m-%d")
    end_date = Date + timedelta(days=+int(deltaDate))
    dateStr=end_date.strftime("%Y-%m-%d")

    strFile='./results/history_'+state+str(version)+'.csv'
    if os.path.isfile(strFile):
        totLines=rawincount(strFile)
    else:
        totLines=-1
    
    #state, loss, start_date, predict_range,s_0, e_0, a_0, i_0, r_0, d_0, startNCases, ratio, weigthCases, weigthRecov, cleanRecovered, version, savedata=True):
    
    learner = L.Learner(state,dateStr, 
                                     predict_range,s0, e0, a0, i0, r0, d0, 
                                     startNCases, ratio, wcases, wrec,
                                     cleanRecovered=cleanRecovered, \
                                     version=version, savedata=False)
    results=learner.train()


    strSave='{}, {}, {}, '.format(totLines+1,state, abs(results[1]))
    strSave=strSave+', '.join(map(str,point))
    strSave=strSave+', '+', '.join(map(str,results[0]))
    append_new_line('./results/history_'+state+str(version)+'.csv', strSave)  

    return results

In [7]:
states=['SP','ES','MG','RJ','PE','CE','AM']
allStates=True
version="10"
ray.shutdown()
ray.init(num_cpus=7)

optimal=[]
if allStates:
    i=0
    for state in states:
        strFile='./results/history_'+state+version+'.csv'
        if os.path.isfile(strFile):
            os.remove(strFile)
        query = dfparam.query('state == "{}"'.format(state)).reset_index()
        parameters = np.array(query.iloc[:, 2:])[0]
        startdate,predict_range,s0,e0,a0,i0,r0,d0,startNCases,ratio,wcases,wrec = parameters
        runID= opt.remote(state,e0,a0,r0,d0,startdate,startNCases, \
                                                        ratio, predict_range, version)
        optimal.append(runID)
        i+=1
        
else:
    state = "ES" 
    #remove previous history file
    strFile='./results/history_'+state+version+'.csv'
    if os.path.isfile(strFile):
        os.remove(strFile)
    query = dfparam.query('state == "{}"'.format(state)).reset_index()
    parameters = np.array(query.iloc[:, 2:])[0]
    startdate,predict_range,s0,e0,a0,i0,r0,d0,startNCases,ratio,wcases,wrec = parameters
    runID= opt.remote(state,e0,a0,r0,d0,startdate,startNCases, \
                                                    ratio, predict_range, version)
    optimal.append(runID)
    



2020-06-08 19:19:38,554	INFO resource_spec.py:205 -- Starting Ray with 20.02 GiB memory available for workers and up to 10.01 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).


NameError: name 'pt' is not defined

In [8]:


for i in range(0,len(states)):
    state=states[i]
    query = dfparam.query('state == "{}"'.format(state)).reset_index()
    parameters = np.array(query.iloc[:, 2:])[0]
    startdate,predict_range,s0,e0,a0,i0,r0,d0,startNCases,ratio,wcases,wrec = parameters
    Date = datetime.strptime(startdate, "%Y-%m-%d")
    end_date = Date + timedelta(days=+int(optimal[i][1]))
    dateStr=end_date.strftime("%Y-%m-%d")
    print("S0 = {}\n".format(optimal[i][0]))
    print("Delta Date Days = {}\n".format(optimal[i][1]))   
    print("I0 = {}\n".format(optimal[i][2]))
    print("wcases = {}\n".format(optimal[i][3]))
    print("wrec = {}\n".format(optimal[i][4]))
    j = query['index'].values[0]
    dfparam.at[j, "s0"] = optimal[i][0]
    dfparam.at[j, "i0"] = optimal[i][2]
    dfparam.at[j, "start-date"] = dateStr
    dfparam.at[j, "WCASES"] = optimal[i][3]
    dfparam.at[j, "WREC"] = optimal[i][4]

IndexError: list index out of range

In [9]:
dfparam


,state,start-date,prediction-range,s0,e0,a0,i0,r0,d0,START,RATIO,WCASES,WREC
0,SP,2020-03-15,200,8000000.0,0,0,800,300,250,100,0.15,0.55,0.01
1,ES,2020-04-01,200,475000.0,0,0,50,250,50,100,0.15,0.65,0.01
2,MG,2020-04-01,200,1100000.0,0,0,200,250,40,100,0.15,0.55,0.01
3,RJ,2020-03-20,200,700000.0,0,0,800,250,50,100,0.15,0.50,0.01
4,CE,2020-03-20,200,800000.0,0,0,800,250,50,100,0.15,0.50,0.01
5,PE,2020-03-20,200,700000.0,0,0,800,250,100,100,0.15,0.50,0.01
6,AM,2020-03-20,200,700000.0,0,0,800,250,100,100,0.15,0.50,0.01


In [10]:
dfparam.to_csv("data/param_optimized_BruteForce.csv", sep=",", index=False)